https://keras.io/examples/nlp/lstm_seq2seq/

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
with open('fra.txt', 'r') as f:
    text_data = f.read().split('\n')
    
text_data = text_data[:10000]

In [ ]:
english_sentences = []
french_sentences = []
english_characters = set()
french_characters = set()

for idx, val in enumerate(text_data):
    eng_data, fre_data, _ = val.split('\t')
    english_sentences.append(eng_data)
    for char in eng_data:
        if char not in english_characters:
            english_characters.add(char)
    
    fre_data = '\t' + fre_data + '\n'
    french_sentences.append(fre_data)
    for char in fre_data:
        if char not in french_characters:
            french_characters.add(char)

In [ ]:
encoder_char_len = len(english_characters)
decoder_char_len = len(french_characters)

encoder_chars = sorted(english_characters)
decoder_chars = sorted(french_characters)

max_encoder_sequence_len = max([len(seq) for seq in english_sentences])
max_decoder_sequence_len = max([len(seq) for seq in french_sentences])

In [ ]:
input_encoder_data = np.zeros((len(english_sentences), max_encoder_sequence_len, encoder_char_len), dtype='float32')
input_decoder_data = np.zeros((len(french_sentences), max_decoder_sequence_len, decoder_char_len), dtype='float32')
target_decoder_data = np.zeros((len(french_sentences), max_decoder_sequence_len, decoder_char_len), dtype='float32')

In [ ]:
encoder_input_index = dict([(char, idx) for idx, char in enumerate(encoder_chars)])
decoder_input_index = dict([(char, idx) for idx, char in enumerate(decoder_chars)])

In [ ]:
assert len(english_sentences) == len(french_sentences)

In [ ]:
sentence_break_space = ' '

In [ ]:
for row, (encoder_data, decoder_data) in enumerate(zip(english_sentences, french_sentences)):
    
    for col, char in enumerate(encoder_data):
        input_encoder_data[row, col, encoder_input_index[char]] = 1.0
    input_encoder_data[row, col+1:, encoder_input_index[sentence_break_space]] = 0
    
    for col, char in enumerate(decoder_data):
        input_decoder_data[row, col, decoder_input_index[char]] = 1.0
        
        if col > 0:
            target_decoder_data[row, col-1, decoder_input_index[char]] = 1.0
            
    input_decoder_data[row, col+1:, decoder_input_index[sentence_break_space]] = 1.0
    target_decoder_data[row, col:, decoder_input_index[sentence_break_space]] = 1.0

In [ ]:
encoder_inputs = tf.keras.Input(shape=(None, encoder_char_len))
encoder_lstm = tf.keras.layers.LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = tf.keras.Input(shape=(None, decoder_char_len))
decoder_lstm = tf.keras.layers.LSTM(256, return_state=True, return_sequences=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state= encoder_states)

decoder_dense = tf.keras.layers.Dense(decoder_char_len, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = tf.keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 71)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, 256), (None, 335872      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, None, 256),  358400      input_2[0][0]                    
                                                                 lstm[0][1]                   

In [ ]:
model.compile(optimizer='rmsprop', metrics=['accuracy'], loss='categorical_crossentropy')

model.fit([input_encoder_data, input_decoder_data], target_decoder_data, batch_size=64, epochs=100, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 37s 47ms/step - loss: 1.5104 - accuracy: 0.7019 - val_loss: 1.0459 - val_accuracy: 0.7101
Epoch 2/100
125/125 [==============================] - 4s 35ms/step - loss: 0.8803 - accuracy: 0.7602 - val_loss: 0.8359 - val_accuracy: 0.7726
Epoch 3/100
125/125 [==============================] - 4s 35ms/step - loss: 0.7139 - accuracy: 0.8053 - val_loss: 0.7176 - val_accuracy: 0.7948
Epoch 4/100
125/125 [==============================] - 4s 35ms/step - loss: 0.5973 - accuracy: 0.8272 - val_loss: 0.6315 - val_accuracy: 0.8161
Epoch 5/100
125/125 [==============================] - 4s 35ms/step - loss: 0.5390 - accuracy: 0.8427 - val_loss: 0.5983 - val_accuracy: 0.8238
Epoch 6/100
125/125 [==============================] - 4s 35ms/step - loss: 0.4978 - accuracy: 0.8538 - val_loss: 0.5665 - val_accuracy: 0.8337
Epoch 7/100
125/125 [==============================] - 4s 35ms/step - loss: 0.4731 - accuracy: 0.8600 - val_loss: 0.5404 - val_accuracy

In [ ]:
reverse_encoder_input_index = dict([(idx, char) for char, idx in encoder_input_index.items()])
reverse_decoder_input_index = dict([(idx, char) for char, idx in decoder_input_index.items()])

In [ ]:
encoder_model = tf.keras.Model(encoder_inputs, encoder_states)

decoder_state_input_h = tf.keras.Input(shape=(256,), name='input_3')
decoder_state_input_c = tf.keras.Input(shape=(256,), name='input_4')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, decoder_state_output_h, decoder_state_output_c = decoder_lstm(decoder_inputs, initial_state= decoder_states_inputs)

decoder_output_states = [decoder_state_output_h, decoder_state_output_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = tf.keras.Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_output_states)

In [ ]:
def decode_sequence(input_seq):

  encoder_context_out = encoder_model.predict(input_seq)

  target_seq = np.zeros((1, 1, decoder_char_len))
  target_seq[0, 0, decoder_input_index['\t']] = 1.0

  decoded_sentence = ''
  stop_condition = False

  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + encoder_context_out)

    sample_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = reverse_decoder_input_index[sample_token_index]
    decoded_sentence += sampled_char

    if sampled_char == '\n' or len(decoded_sentence) > max_decoder_sequence_len:
      stop_condition = True

    target_seq = np.zeros((1, 1, decoder_char_len))
    target_seq[0, 0, sample_token_index] = 1.0

    encoder_context_out = [h, c]
    
  return decoded_sentence


In [ ]:
for idx in range(20):

  eng_sent = english_sentences[idx]
  print(f'English Sentence: {eng_sent}')

  input_seq = input_encoder_data[idx:idx+1]
  #print(input_seq)
  french_sent = decode_sequence(input_seq)
  print(f'Translated French Sentence: {french_sent}')

English Sentence: Go.
Translated French Sentence: Va !

English Sentence: Go.
Translated French Sentence: Va !

English Sentence: Go.
Translated French Sentence: Va !

English Sentence: Hi.
Translated French Sentence: Salut !

English Sentence: Hi.
Translated French Sentence: Salut !

English Sentence: Run!
Translated French Sentence: Fuyons !

English Sentence: Run!
Translated French Sentence: Fuyons !

English Sentence: Run!
Translated French Sentence: Fuyons !

English Sentence: Run!
Translated French Sentence: Fuyons !

English Sentence: Run!
Translated French Sentence: Fuyons !

English Sentence: Run!
Translated French Sentence: Fuyons !

English Sentence: Run!
Translated French Sentence: Fuyons !

English Sentence: Run!
Translated French Sentence: Fuyons !

English Sentence: Run.
Translated French Sentence: Fuyons !

English Sentence: Run.
Translated French Sentence: Fuyons !

English Sentence: Run.
Translated French Sentence: Fuyons !

English Sentence: Run.
Translated French Se